In [13]:
import sys
import os
import pprint
import pandas as pd
from pandarallel import pandarallel

In [3]:
path_to_talla = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

In [4]:
if path_to_talla not in sys.path:
    sys.path.append(path_to_talla)

In [5]:
from FinABSA.FinABSA import ABSA

In [10]:
news = pd.read_csv('../data/guardian_financial_news_master.csv')
test = news.iloc[0:10]

In [11]:
test

,url,title,pub_date,summary,body
0,https://www.theguardian.com/technology/2025/fe...,AI tools used for child sexual abuse images ta...,2025-02-01T22:00:53Z,UK will be first country to bring in tough new...,Britain is to become the first country to intr...
1,https://www.theguardian.com/technology/2025/fe...,"As DeepSeek upends the AI industry, one group ...",2025-02-01T19:00:49Z,Recruiting top graduates could stop other coun...,One Australian company has discouraged staff f...
2,https://www.theguardian.com/technology/2025/fe...,Was this the week DeepSeek started the slow un...,2025-02-01T16:51:23Z,The cheap Chinese chatbot has stunned tech gia...,"At 2.16pm California time last Sunday, the US ..."
3,https://www.theguardian.com/technology/2025/fe...,AI is not just powerful. What’s really worryin...,2025-02-01T16:00:46Z,The AI startup has upended the industry by dev...,Nothing cheers up a tech columnist more than t...
4,https://www.theguardian.com/technology/2025/fe...,"DeepSeek, ChatGPT, Grok … which is the best AI...",2025-02-01T12:00:42Z,Chatbots we tested can write a mean sonnet and...,ChatGPT and its owners must have hoped it was ...
5,https://www.theguardian.com/business/2025/feb/...,‘Dark and dingy’ or ‘a real loss’? Shoppers re...,2025-02-01T10:00:38Z,Stevenage is one of 500 UK towns where threate...,Just two shop floor staff are on duty and shop...
6,https://www.theguardian.com/business/2025/feb/...,Small UK businesses complain of being caught u...,2025-02-01T07:00:35Z,New safety regulations have led SMEs to stop s...,Small businesses are warning they have had to ...
7,https://www.theguardian.com/technology/2025/ja...,Italian investigative journalist targeted on W...,2025-01-31T23:46:26Z,"Francesco Cancellato, whose reporting exposes ...",An Italian investigative journalist who is kno...
8,https://www.theguardian.com/environment/2025/j...,‘We’d go absolutely nuts’: PM warned of Labour...,2025-01-31T17:59:59Z,Exclusive: MPs and ministers say they would op...,Senior Labour figures are warning of a serious...
9,https://www.theguardian.com/technology/2025/ja...,Ed Ferrett obituary,2025-01-31T17:45:35Z,<strong>Other lives:</strong> Engineering and ...,"My father, Ed Ferrett, who has died aged 83, w..."


In [15]:
def sentiment_model(text, absa):
    o = absa.run_absa(input_str = text)
    return o

In [ ]:
#RUN CODE WITHOUT PANDARALLEL (CAUSING ISSUES)
pandarallel.initialize(progress_bar=True)

print("\n--- Running with pandarallel ---")

# 1. Apply the model in parallel. 
# We just change .apply() to .parallel_apply()
absa = ABSA()
test['sentiment_results'] = test['body'].parallel_apply(sentiment_model, args=(absa,))

# 2. The un-nesting logic is identical to the previous pandas solution.
# This part is very fast and doesn't need to be parallelized.
long_series = test.set_index('date')['sentiment_results'].apply(pd.Series).stack()

# 3. Convert the Series to a DataFrame and fix column names
long_df = long_series.reset_index()
long_df.columns = ['date', 'entity', 'details']

# 4. Split the 'details' column
long_df['classification'] = long_df['details'].apply(lambda x: x['classification_output'])
long_df['softmax_scores'] = long_df['details'].apply(lambda x: x['logits'])

# 5. Drop the intermediate 'details' column
final_df_pandarallel = long_df.drop(columns='details')

print("\n--- Final DataFrame (from pandarallel) ---")
print(final_df_pandarallel)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/

--- Running with pandarallel ---
2025-10-29 12:47:01,806 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [ ]:
absa = ABSA()
o = absa.run_absa(input_str = text)
pprint.pprint(o)

2025-10-14 14:46:06,571 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


Token indices sequence length is longer than the specified maximum sequence length for this model (1027 > 512). Running this sequence through the model will result in indexing errors


{'AI': {'classification_output': 'POSITIVE',
        'logits': {'negative': 0.005481464322656393,
                   'neutral': 0.17708371579647064,
                   'positive': 0.8174348473548889}},
 'Bell': {'classification_output': 'NEUTRAL',
          'logits': {'negative': 0.0036934372037649155,
                     'neutral': 0.7652527689933777,
                     'positive': 0.23105382919311523}},
 'BlackRock Investment Institute': {'classification_output': 'NEUTRAL',
                                    'logits': {'negative': 0.0007032313151285052,
                                               'neutral': 0.9853926301002502,
                                               'positive': 0.013904097490012646}},
 'Cessna': {'classification_output': 'NEUTRAL',
            'logits': {'negative': 0.004992321133613586,
                       'neutral': 0.757658064365387,
                       'positive': 0.23734962940216064}},
 'Charmin': {'classification_output': 'NEUTRAL',
        

In [8]:
o.keys()

dict_keys(['Netflix', 'Oracle', 'Dow Jones Industrial Average', 'Nasdaq', 'Treasury', 'Procter & Gamble', 'Charmin', 'Crest', 'Pampers', 'Stargate', 'AI', 'OpenAI', 'SoftBank', 'SoftBank Group Corp.’s', 'Nvidia', 'Textron', 'Bell', 'Cessna', 'BlackRock Investment Institute'])

import torch
path_to_finentity = os.path.join(path_to_talla, 'FinEntity')

if path_to_finentity not in sys.path:
    sys.path.append(path_to_finentity)


# Now, open the file using the full path
model_file_path = os.path.join(path_to_finentity, 'model_bert_crf')
import pickle
import io
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            # Intercept the problematic function and rewrite it to use CPU
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu', weights_only=False)
        else:
            return super().find_class(module, name)

# --- Load the model using the custom unpickler ---
print(f"Loading model from: {model_file_path}")
try:
    with open(model_file_path, 'rb') as f:
        model = CPU_Unpickler(f).load()
    
    print("Model loaded successfully using custom CPU unpickler!")

except Exception as e:
    print(f"An error occurred: {e}")

import json
from sequence_aligner.dataset import PredictDatasetCRF
from sequence_aligner.containers import TraingingBatch,PredictBatch
from transformers import BertTokenizerFast
from sequence_aligner.labelset import LabelSet
from util.train import predict,predict4news
from torch import cuda
import warnings
warnings.filterwarnings('ignore')

f3 = open(path_to_finentity+'/news_data/20230105.json', 'r', encoding='utf8')
content3 = f3.read() 
raw = json.loads(content3)
data = [example for example in raw if example["content"]!= None and example["content"]!=[]]


save_list = []
tokenizer = BertTokenizerFast.from_pretrained('yiyanghkust/finbert-pretrain')
label_set = LabelSet(labels=["Neutral", "Positive", "Negative"])  # label in this dataset
model.to('cpu')

'''
for item in data:
    predict4news(model,item,device,tokenizer,label_set,save_list)
'''